---
# Topological Cluster Statistics (TCS)

---

## Notebook 1: loading PALM results

---

This notebook contains scripts to load the results of running TCS and cluster-based statistics using PALM.



---

### Packages and basic functions


---

Loading required packages


In [1]:
import os
import numpy as np
import nibabel as nib
from tqdm.notebook import tqdm


---

Basic functions


In [2]:
def ensure_dir(file_name):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    return file_name


def write_np(np_obj, file_path):
    with open(file_path, 'wb') as outfile:
        np.save(outfile, np_obj)


def load_np(file_path):
    with open(file_path, 'rb') as infile:
        return np.load(infile)

---

### results from running PALM


---

The script below reads the outputs generated by palm to analyze the differences between traditional cluster extent statistic and the topological cluster statistic. The methods have been repeated (500 repetitions) across a wide range of sample sizes (10, 20, 40, 80, 160, 320). For any repition of a sample size the effect is clustered (cdt z = {3.3, 2.8, 2.6, 2.0, 1.6}, which is respectively equivalent to a two-sided test with p = {0.001, 0.005, 0.01, 0.05, 0.1}) with spatial or topological + spatial neighborhood. A permutation of 1000 sign flips is used to determine the minimum cluster size that is deemed significant after clustering (analyses conducted in PALM).

The output is saved in a binary `.npy` format which can be read efficiently by Python's NumPy library.

In [ ]:
%%time

# list of all tasks and the cope number related to each selected contrast
tasks = {
    'EMOTION': '3',  # faces - shapes
    'GAMBLING': '6',  # reward - punish
    'RELATIONAL': '4',  # rel - match
    'SOCIAL': '6',  # tom - random
    'WM': '20',  # face - avg
}
# Number of random repetitions
repetitions = 500
# Different sample sizes tested
sample_sizes = [10, 20, 40, 80, 160, 320]
# Different cluster defining thresholds
cdts = [3.3, 2.8, 2.6, 2.0, 1.6]
# Number of brainordinates in a cifti file
Nv = 91282
# Base directory where files are stored at
base_dir='/data/netapp01/work/sina/structural_clustering/PALM_revision_1'

for cdt in tqdm(cdts, desc="CDT loop", leave=True):
    for task in tqdm(tasks, desc="Tasks loop", leave=False):
        for sample_size in tqdm(sample_sizes, desc="Sample size loop", leave=False):
            uncorrected_tstat = np.zeros((repetitions, Nv))
            spatial_cluster_corrected_tstat = np.zeros((repetitions, Nv))
            topological_cluster_corrected_tstat = np.zeros((repetitions, Nv))
            valid_repetitions = set()

            for repetition in tqdm(range(repetitions), desc="Repetition loop", leave=False):
                try:
                    # locations of outputs generated by PALM
                    spatial_dir = f'{base_dir}/local_adjacency/{sample_size}_samples/run_{repetition}'
                    topology_dir = f'{base_dir}/hybrid_adjacency_clnorm/{sample_size}_samples/run_{repetition}'

                    # load cifti tstats generated by PALM
                    uncorrected_tstat[repetition] = nib.load(f'{spatial_dir}/{task}_{cdt}_conn_tstat.dscalar.nii').get_fdata()
                    spatial_cluster_corrected_tstat[repetition] = nib.load(f'{spatial_dir}/{task}_{cdt}_clustertopo_tstat_fwep.dscalar.nii').get_fdata()
                    topological_cluster_corrected_tstat[repetition] = nib.load(f'{topology_dir}/{task}_{cdt}_clustertopo_tstat_fwep.dscalar.nii').get_fdata()
                    valid_repetitions.add(repetition)
                except:
                    # in case there were errors in PALM execution, discard that repetition
                    uncorrected_tstat[repetition] = np.nan
                    spatial_cluster_corrected_tstat[repetition] = np.nan
                    topological_cluster_corrected_tstat[repetition] = np.nan
                    valid_repetitions.discard(repetition)

            # store loaded values in NPY binaries
            write_np(
                uncorrected_tstat,
                ensure_dir(f'{base_dir}/summary/uncorrected_tstat_{task}_{sample_size}_samples_{cdt}_CDT.npy'),
            )
            write_np(
                spatial_cluster_corrected_tstat,
                ensure_dir(f'{base_dir}/summary/spatial_cluster_corrected_tstat_{task}_{sample_size}_samples_{cdt}_CDT.npy'),
            )
            write_np(
                topological_cluster_corrected_tstat,
                ensure_dir(f'{base_dir}/summary/topological_cluster_corrected_tstat_{task}_{sample_size}_samples_{cdt}_CDT.npy'),
            )


CDT loop:   0%|          | 0/5 [00:00<?, ?it/s]

Tasks loop:   0%|          | 0/5 [00:00<?, ?it/s]

Sample size loop:   0%|          | 0/6 [00:00<?, ?it/s]

Repetition loop:   0%|          | 0/500 [00:00<?, ?it/s]

Repetition loop:   0%|          | 0/500 [00:00<?, ?it/s]

...

Sample size loop:   0%|          | 0/6 [00:00<?, ?it/s]

Repetition loop:   0%|          | 0/500 [00:00<?, ?it/s]

Repetition loop:   0%|          | 0/500 [00:00<?, ?it/s]

Repetition loop:   0%|          | 0/500 [00:00<?, ?it/s]

Repetition loop:   0%|          | 0/500 [00:00<?, ?it/s]